In [58]:
import matplotlib.pyplot as plt
import matplotlib as pmimg
import numpy as np
import skimage
from skimage import color
from skimage import data
from skimage import io
from tqdm import tqdm
import cv2

import os 
dir_path = os.getcwd()

In [59]:
def oneHot(imgTitle):
    animalLabel = imgTitle.split('.')[0][-3:]
    return animalLabel

In [60]:
def createTrainSet():
    trainingSet = []
#     data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/practice-train'
    data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/alternate_data/train'
    for imgLabel in tqdm(os.listdir(data_dir)):
        animalLabel = oneHot(imgLabel)
        #dog = [1, 0], cat = [0,1]
        oneHotLabel = []
        if animalLabel == 'dog':
            oneHotLabel = [1, 0]
        elif animalLabel == 'cat':
            oneHotLabel = [0, 1]
        else:
            print('nada')
            continue
        imgPath = os.path.join(data_dir, imgLabel)
        imgObj = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
        imgObj = cv2.resize(imgObj, (51, 51))

        trainingSet.append([imgObj, oneHotLabel])
    return trainingSet

train = createTrainSet()

100%|██████████| 25000/25000 [00:23<00:00, 1066.44it/s]


In [61]:
# def max_pool(img, filter, padding, stride, depth):
#     outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
#     print('OUTPUT DIM IS:' + str(outputDim))
#     resultArr = np.zeros(((depth), (outputDim), (outputDim)), dtype=int)
#     filterLength = len(filter)
#     for zIdx in range(depth):
#         for yIdx in range(1, outputDim-1):
#             for xIdx in range(1, outputDim-1):
#                 window = np.zeros((filterLength, filterLength), dtype=int)

#                 windowYIdx = 0 + yIdx
#                 while windowYIdx < (len(filter) + yIdx):
#                     windowXEnd = xIdx + len(filter)
#                     currRow = np.array(img[windowYIdx][xIdx:windowXEnd])
#                     window[windowYIdx - yIdx] = currRow
#                     windowYIdx += 1

#                 filteredWindow = window * filter
#                 nodeValue = np.amax(filteredWindow)
#                 resultArr[zIdx][yIdx][xIdx] = nodeValue
    
#     return resultArr

def max_pool(img, filter, padding='same', stride=1, depth=1):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
    padded_img = np.pad(img, ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    
    filterLength = len(filter)

    for zIdx in range(depth):
        for yIdx in range(padded_img.shape[0] - (2 * padding)):
            for xIdx in range(padded_img.shape[1] - (2 * padding)):
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.amax(filteredWindow)
                resultArr[zIdx][yIdx][xIdx] = nodeValue
    #Boom!

In [62]:
def relu(imgMatrix):
    filterMask = imgMatrix > 0
    reluMatrix = imgMatrix * filterMask
    return reluMatrix

In [63]:
def convolve_2d(img, filter, padding='same', stride=1, depth=1):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
    padded_img = np.pad(img, ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    
    filterLength = len(filter)

    for zIdx in range(depth):
        for yIdx in range(padded_img.shape[0] - (2 * padding)):
            for xIdx in range(padded_img.shape[1] - (2 * padding)):
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.average(filteredWindow)
                resultArr[zIdx][yIdx][xIdx] = nodeValue
    #Boom!
    return resultArr

    
def createConvolutionalLayer(filtersArr, img):
    convFilterArr = []
    for filter in filtersArr:
        convResult = convolve_2d(img, filter, depth=1)
        convFilterArr.append(convResult)
    return convFilterArr    

side_detector_layer = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])

reverse_side_detector_layer = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

top_bottom_detector_layer = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, 1]
])

angle_detector_layer = np.array([
    [0, 1, 1],
    [-1, 0, 1],
    [-1, -1, 0]
])


top_detector_layer = np.array([
    [1, 1, 1],
    [-1, -1, -1],
    [-1, -1, -1]
])


x_detector_layer = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, 1]
])

right_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1]
]);

left_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0]
]);

small_angle_detector_layer = np.array([
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
])

filters = [
    side_detector_layer,
    reverse_side_detector_layer,
    small_angle_detector_layer,
    top_bottom_detector_layer,
    angle_detector_layer,
    top_detector_layer,
    left_knight_detector_layer,
    right_knight_detector_layer,
    x_detector_layer
]

animal = 4773
imageOne = train[animal][0]
convolved = convolve_2d(imageOne, side_detector_layer)
conv_layer = createConvolutionalLayer(filters, imageOne)

def train_conv_layer(imgSet):
    x = 0
    for img in tqdm(imgSet):
        imgSet[x][0] = createConvolutionalLayer(filters, img[0])
        x += 1
    return imgSet

convolved_img_set = train_conv_layer(train[0:100])
print(convolved_img_set[0][0][0])

100%|██████████| 100/100 [00:52<00:00,  1.92it/s]

[[[  8   9   4 ...  19  21 -34]
  [ 14  13   6 ...  21  28 -51]
  [ 17  12   5 ...  16  21 -51]
  ...
  [  9   7  21 ...   2   3 -16]
  [ 10   4  12 ...   4   3 -17]
  [  5   1   8 ...   2   2 -11]]]


In [64]:
print(convolved_img_set[0][0][0].shape)

(1, 51, 51)


In [66]:
animal = 4773
imageOne = train[animal][0]
convolved = convolve_2d(imageOne, side_detector_layer)
print('==============')
print('=====convolve=======')
print(convolved)
print('=====end convolve=====')
print('==============')
# maxed = max_pool(convolved, side_detector_layer)
print('==============')
print('=====max=======')
# print(maxed)
print('=====end max=====')
print('==============')


=====convolve=======
[[[  3  -1   1 ...   0   0 -35]
  [  4  -3   2 ...   0  -1 -53]
  [  5  -3   1 ...   0  -1 -54]
  ...
  [ 47   1   0 ...   2   2 -14]
  [ 47   1   0 ...  -1   0 -12]
  [ 31   0   0 ...  -4   0  -6]]]
=====end convolve=====
=====max=======
=====end max=====


In [ ]:
# FILTERS



big_edge_detector = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
])


reverse_big_edge_detector = np.array([
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
])

circular_edge_detector = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

horiz_filter = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1],
])
vertical_filter = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])
whole_filter = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
])

In [ ]:
def relu_layer(imgMatrixArr):
    imgDepth = len(imgMatrixArr)
    imgLength = imgMatrixArr[0].shape[0]
    imgWidth = imgMatrixArr[0].shape[1]
    
    idx = 0
    relu_arr = np.random.randint(10, size=(imgDepth, imgLength, imgWidth))
    for imgMat in imgMatrixArr:
        relu_arr[idx] = relu(imgMat)
        idx += 1
    return relu_arr
    

def max_pool_layer(imgArr):
    depth = len(imgArr)
    maxi = max_pool(imgArr[0], vertical_filter, padding=1, stride=1, depth=depth)
    idx = 0
    for img in imgArr:
        print(img)
        imgArr[idx] = max_pool(img, vertical_filter)
    return imgArr

# animal = 4773
# imageOne = train[animal][0]

# convLayer = createConvolutionalLayer(filters, imageOne)
# print(convLayer[0].shape)
    
# relu_matrix = relu_layer(convLayer)
# print(relu_matrix.shape)